# NN Model

In [1]:
import numpy as np
from sklearn.metrics import mean_squared_error as mse 
# local imports
from model_tools import PredictorData
from nn_models import TF_NN_Predictor
from utils import Utils

2024-06-09 11:04:25.636382: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-09 11:04:25.797909: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-09 11:04:26.596430: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-09 11:04:34.667277: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
config = Utils.read_config_for_env(config_path='../config/config.yml')
pred_data = PredictorData(
    config,
    refresh_monthly=False,
    refresh_ts_features=False,
    clean_strategy='olrem_for_all',
    # split_strategy='random',
    # split_strategy='last_months_val',
    split_strategy='months',
    num_lag_mon=3,
    val_ratio=0.2)

Loading data.. Done.
Fixing data schemas.. Done.
Loading /home/onur/WORK/DS/repos/shop_sales_prediction/data/all_ts.parquet


## NN model with 3 Dense layers

In [4]:
tf_nn_predictor_3D = TF_NN_Predictor(
    pred_data=pred_data,
    output_dim=1,
    model_name='3Dense',
    optimizer="adam",
    metrics=['mse'])

In [5]:
tf_nn_predictor_3D.split_transform()

Splitting train-val
Fit-transforming X_train
Transforming X_val


In [6]:
tf_nn_predictor_3D.create_model()

Creating model


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           608 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,753 (10.75 KB)

 Trainable params: 2,753 (10.75 KB)

 Non-trainable params: 0 (0.00 B)

In [7]:
#Train the model
tf_nn_predictor_3D.model.fit(
    tf_nn_predictor_3D.pred_data.X_train,
    tf_nn_predictor_3D.pred_data.y_train,
    epochs=10,
    batch_size=4096,
    validation_data=(tf_nn_predictor_3D.pred_data.X_val, tf_nn_predictor_3D.pred_data.y_val),
    callbacks=tf_nn_predictor_3D.callbacks)

2024-06-09 11:07:58.551714: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 2394360000 exceeds 10% of free system memory.


Epoch 1/10
8119/8119 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.3700 - mse: 0.3700
Epoch 1: val_loss improved from inf to 0.39681, saving model to 3Dense_weights-001--0.39681.keras
8119/8119 ━━━━━━━━━━━━━━━━━━━━ 291s 33ms/step - loss: 0.3700 - mse: 0.3700 - val_loss: 0.3968 - val_mse: 0.3968
Epoch 2/10
8119/8119 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.3025 - mse: 0.3025
Epoch 2: val_loss did not improve from 0.39681
8119/8119 ━━━━━━━━━━━━━━━━━━━━ 274s 32ms/step - loss: 0.3025 - mse: 0.3025 - val_loss: 0.4102 - val_mse: 0.4102
Epoch 3/10
8114/8119 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.2994 - mse: 0.2994
Epoch 3: val_loss did not improve from 0.39681
8119/8119 ━━━━━━━━━━━━━━━━━━━━ 81s 9ms/step - loss: 0.2994 - mse: 0.2994 - val_loss: 0.3993 - val_mse: 0.3993
Epoch 4/10
8118/8119 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.2935 - mse: 0.2935
Epoch 4: val_loss improved from 0.39681 to 0.39385, saving model to 3Dense_weights-004--0.39385.keras
8119/8119 ━━━━━━━━━━━━━━━━━━━━ 39s 5ms

In [8]:
# Make predictions on the training and validation data. 
pred_train = tf_nn_predictor_3D.model.predict_on_batch(pred_data.X_train)
pred_val = tf_nn_predictor_3D.model.predict_on_batch(pred_data.X_val)

# Calculate and print the Root Mean Squared Error (RMSE) for training and validation predictions. 
print("Training RMSE: ", np.sqrt(mse(pred_data.y_train, pred_train)))
print("Validation RMSE: ", np.sqrt(mse(pred_data.y_val, pred_val)))

2024-06-09 11:23:26.593601: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 4788720000 exceeds 10% of free system memory.
2024-06-09 11:23:28.277106: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 2394360000 exceeds 10% of free system memory.
2024-06-09 11:23:28.790014: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 4256640000 exceeds 10% of free system memory.
2024-06-09 11:23:31.149178: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 4256640000 exceeds 10% of free system memory.


Training RMSE:  0.52630854
Validation RMSE:  0.62565887
